In [1]:
import requests as rq
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
all_reviews_dict = {}

In [4]:
url = 'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=&preliminary=on&spoiler=on&p=1'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}
page = rq.get(url, headers=headers)
print(page.status_code)

200


In [ ]:
url_nopage = 'https://myanimelist.net/reviews.php?t=anime&filter_check=&filter_hide=&preliminary=on&spoiler=on&p='
page_num = 0
url = url_nopage + str(page_num)
page = rq.get(url, headers=headers)
review_dict = {}

while page.status_code == 200:
    # print(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    for item in soup.find_all(attrs={"class": "review-element js-review-element"}):
        try:
            title = item(attrs={"class": "title ga-click"})[0].get_text()
            tag = item(attrs={"class": "tags"})[0].get_text()
            review = item(attrs={"class": "text"})[0].get_text()
            rating = item(attrs={"class": "num"})[0].get_text()
            review_dict[review] = (title, tag, rating)
        except:
            continue
    page_num +=1
    url = url_nopage + str(page_num)
    page = rq.get(url, headers=headers)

In [6]:
all_reviews_dict.update(review_dict)

In [7]:
len(list(all_reviews_dict.items()))

242143

In [12]:
# import pickle

# with open('all_reviews_dict.pkl', 'wb') as f:
#     pickle.dump(all_reviews_dict, f)

In [2]:
import pickle

with open('all_reviews_dict.pkl', 'rb') as f:
    all_reviews_dict = pickle.load(f)

In [9]:
df = pd.DataFrame(list(all_reviews_dict.values()), columns=['title', 'tag', 'rating'], index = list(all_reviews_dict.keys()))
df.reset_index(inplace=True)
df.rename(columns={'index': 'review'}, inplace=True)
df

,review,title,tag,rating
0,"\n\n\n\n\n\n Cringe ""comedy"" in...",Amagi Brilliant Park,Not RecommendedPreliminary\n \n...,2
1,\n\n\n\n\n\n So this is the thi...,Higeki no Genkyou to Naru Saikyou Gedou Last B...,Mixed Feelings,5
2,\n\n\n\n\n\n The show is depres...,Sousou no Frieren,RecommendedPreliminary\n \n ...,9
3,"\n\n\n\n\n\n Ugly CGI, bland vi...",Kimetsu no Yaiba,Not Recommended,3
4,\n\n\n\n\n\n Boring. Cliche. La...,Ousama Ranking,Not Recommended,2
...,...,...,...,...
242138,\n\n\n\n\n\n A comedy/romance b...,Ranma ½,Recommended,7
242139,\n\n\n\n\n\n Saiyuki is one of ...,Gensoumaden Saiyuuki,Recommended,9
242140,"\n\n\n\n\n\n First, let me say ...",Hajime no Ippo,Recommended,10
242141,\n\n\n\n\n\n Utawarerumono mana...,Utawarerumono,Recommended,8


In [11]:
df.to_csv('reviews.csv', sep=';', encoding='utf-8')